In [1]:
import warnings
warnings.filterwarnings('ignore')
from pyspark import SparkContext
from datetime import datetime, timedelta

In [5]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
lines = sc.textFile("/Users/halilergul/Desktop/master/fall-23_24/datasets-20231023/DollarDataset.txt")

def parse(line):
    fields = line.split('\t')
    date = fields[1]
    price_str = fields[2].replace('.', '').replace(',', '.') # I remove periods and replace commas with a period
    price = 0 if price_str == '' else float(price_str) # # If the price string is empty, I assign it as 0, else float
    return (date, price)
data = lines.map(parse)

def sort_key(item):
    date_parts = item[0].split('-')
    return f"{date_parts[2]}-{date_parts[1]}-{date_parts[0]}"
sorted_data = data.sortBy(sort_key) # Now I sort the data by date

In [6]:
shifted_data = sorted_data.zipWithIndex().map(lambda x: (x[1]-1, x[0]))  # I shift index by one position to compare each day with its preceding day
shifted_data.take(5) # zipwithindex adds an index to each element of the RDD

[(-1, ('02-01-1950', 2.8)),
 (0, ('03-01-1950', 2.8)),
 (1, ('04-01-1950', 2.8)),
 (2, ('05-01-1950', 2.8)),
 (3, ('06-01-1950', 2.8))]

In [8]:
indexed_data = sorted_data.zipWithIndex().map(lambda x: (x[1], x[0])) # I add an index to each element of the RDD
paired_data = indexed_data.join(shifted_data).values() # This will give me pairs of (current_day, previous_day)

def percentage_change(pair):
    current_day_price = pair[0][1]
    previous_day_price = pair[1][1]
    if previous_day_price == 0:  # This is to avoid division by zero
        return 0
    return ((current_day_price - previous_day_price) / previous_day_price) * 100

percentage_data = paired_data.map(lambda x: (x, percentage_change(x)))

sorted_percentage_data = percentage_data.sortBy(lambda x: x[1], ascending=False) # I sort the data by percentage change in descending order
top_5_increases = sorted_percentage_data.take(5) # I take the top 5 increases

for record in top_5_increases:
    date = record[0][0][0]
    increase = record[1]
    print(f"On {date}, the increase was {increase:.2f}%")

On 31-12-2004, the increase was 100156616.42%
On 26-02-2001, the increase was 13.39%
On 30-10-2008, the increase was 12.67%
On 22-03-1991, the increase was 12.23%
On 08-04-1994, the increase was 9.86%
